# Base 3D CNN Model

## Setup

In [1]:
# !pip install -q tensorflow tensorflow-metal scikit-learn ipywidgets

In [2]:
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import tensorflow as tf
import datetime
import pandas as pd

In [3]:
patch_size_pixels = 64
model_input_width = 6  # number of images before the event
buffer_size = 100
# num_classes = 6
batch_size = 8


datasets_folder = os.path.join("E:\\Workspace\\Thesis_dataset", "illinois_dataset")
# dataset_folder_name = "Quercus_germany_200_stations"

In [4]:
dataset_folder_path = datasets_folder
os.makedirs(dataset_folder_path, exist_ok=True)

record_id_column_name = 'record_id'
label_column_name = "ordered_phase_id"

train_timeseries_file_path = os.path.join(dataset_folder_path, "train_data.npz")
print(train_timeseries_file_path)
train_label_file_path = os.path.join(dataset_folder_path, "train.csv")
print(train_label_file_path)
print("-" * 60)
# val_timeseries_file_path = os.path.join(dataset_folder_path, "val_data.npz")
# print(val_timeseries_file_path)
# val_label_file_path = os.path.join(dataset_folder_path, "val.csv")
# print(val_label_file_path)
# print("-" * 60)
# test_timeseries_file_path = os.path.join(dataset_folder_path, "test_data.npz")
# print(test_timeseries_file_path)
# test_label_file_path = os.path.join(dataset_folder_path, "test.csv")
# print(test_label_file_path)

E:\Workspace\Thesis_dataset\illinois_dataset\train_data.npz
E:\Workspace\Thesis_dataset\illinois_dataset\train.csv
------------------------------------------------------------


In [6]:
npz_file = np.load(train_timeseries_file_path, mmap_mode="r")
first_array_key = next(iter(npz_file))
timeseries_shape = npz_file[first_array_key].shape
timeseries_shape

(6, 64, 64, 15)

## Create the datasets

Create the train, validation, and test dataset

In [15]:
label_csv_path = r"E:\Workspace\Thesis_dataset\illinois_dataset\train.csv"
test_label_file_path = r"E:\Workspace\Thesis_dataset\illinois_dataset\test.csv"
record_id_column_name = "id"
label_column_name = "label"
timeseries_npz_path = r"E:\Workspace\Thesis_dataset\illinois_dataset\train_data.npz"
test_timeseries_file_path = r"E:\Workspace\Thesis_dataset\illinois_dataset\train_data.npz"

In [82]:
def data_generator(
    label_csv_path, record_id_column_name, label_column_name, timeseries_npz_path
):

    # Check if the inputs are a byte string and decode it to a regular string if necessary
    if isinstance(label_csv_path, bytes):
        label_csv_path = label_csv_path.decode("utf-8")
    if isinstance(timeseries_npz_path, bytes):
        timeseries_npz_path = timeseries_npz_path.decode("utf-8")
    if isinstance(record_id_column_name, bytes):
        record_id_column_name = record_id_column_name.decode("utf-8")
    if isinstance(label_column_name, bytes):
        label_column_name = label_column_name.decode("utf-8")

    # Load the labels CSV
    labels_df = pd.read_csv(label_csv_path)

    # Load the npz file
    with np.load(timeseries_npz_path, allow_pickle=True) as npz_file:
        for _, row in labels_df.iterrows():
            record_id = str(
                row[record_id_column_name]
            )  # Ensure record_id is treated as a string
            target = row[label_column_name]
            if record_id in npz_file.files:
                # Extract the time series data using record_id
                time_series = npz_file[record_id]
                # print(record_id, target)
                yield time_series, target


# Determine the output types
output_types = (tf.float32, tf.int64)

# Determine the output shapes
output_shapes = (timeseries_shape, ())  #  ((6, 11, 64, 64), ())

# Create train dataset
train_dataset = tf.data.Dataset.from_generator(
    data_generator,  # Generator function
    args=(
        train_label_file_path,
        record_id_column_name,
        label_column_name,
        train_timeseries_file_path,
    ),  # Arguments to pass to the generator
    output_types=output_types,
    output_shapes=output_shapes,
)
train_dataset = (
    train_dataset.shuffle(buffer_size=buffer_size)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

# # Create validation dataset
# val_dataset = tf.data.Dataset.from_generator(
#     data_generator,  # Generator function
#     args=(
#         val_label_file_path,
#         record_id_column_name,
#         label_column_name,
#         val_timeseries_file_path,
#     ),  # Arguments to pass to the generator
#     output_types=output_types,
#     output_shapes=output_shapes,
# )
# val_dataset = (
#     val_dataset.shuffle(buffer_size=buffer_size)
#     .batch(batch_size)
#     .prefetch(tf.data.AUTOTUNE)
# )

# Create test dataset
test_dataset = tf.data.Dataset.from_generator(
    data_generator,  # Generator function
    args=(
        test_label_file_path,
        record_id_column_name,
        label_column_name,
        test_timeseries_file_path,
    ),  # Arguments to pass to the generator
    output_types=output_types,
    output_shapes=output_shapes,
)
test_dataset = test_dataset.shuffle(buffer_size=buffer_size).batch(batch_size)

Print the shape of the datasets

In [83]:
for data in train_dataset.take(1):  # Print the first batch of data
    print(data)


UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} KeyError: 'label'
Traceback (most recent call last):

  File "c:\Users\Surya Naganathan\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\indexes\base.py", line 3790, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

  File "index.pyx", line 152, in pandas._libs.index.IndexEngine.get_loc

  File "index.pyx", line 181, in pandas._libs.index.IndexEngine.get_loc

  File "pandas\_libs\hashtable_class_helper.pxi", line 7080, in pandas._libs.hashtable.PyObjectHashTable.get_item

  File "pandas\_libs\hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item

KeyError: 'label'


The above exception was the direct cause of the following exception:


Traceback (most recent call last):

  File "c:\Users\Surya Naganathan\AppData\Local\Programs\Python\Python312\Lib\site-p

In [64]:
# Function to print the shapes of the dataset
def print_dataset_shapes(dataset, dataset_name="Dataset"):
    print(f"--- {dataset_name} ---")
    for data, label in dataset.take(1):  # Take a single batch from the dataset
        print(f"Shape of data: {data.shape}")
        print(f"Shape of label: {label.shape}")

# Print shapes for train, validation, and test datasets
print_dataset_shapes(train_dataset, "Train Dataset")
# print_dataset_shapes(val_dataset, "Validation Dataset")
print_dataset_shapes(test_dataset, "Test Dataset")


--- Train Dataset ---
Shape of data: (8, 6, 64, 64, 15)
Shape of label: (8,)
--- Test Dataset ---
Shape of data: (8, 6, 64, 64, 15)
Shape of label: (8,)


Normalization

In [65]:
def compute_normalization_stats(dataset):
    """
    Compute the mean and standard deviation of the dataset.
    
    Args:
    dataset: A TensorFlow Dataset from which the statistics should be computed.

    Returns:
    mean: Mean of the dataset.
    stddev: Standard deviation of the dataset.
    """
    # Initialize variables to compute mean and variance
    total_sum = 0
    total_squared_sum = 0
    total_count = 0

    # Iterate through the dataset to compute the sum and squared sum
    for data, _ in dataset:
        total_sum += tf.reduce_sum(data, axis=0)
        total_squared_sum += tf.reduce_sum(tf.square(data), axis=0)
        total_count += tf.cast(tf.shape(data)[0], tf.float32)

    # Compute the mean and variance
    mean = total_sum / total_count
    variance = (total_squared_sum / total_count) - tf.square(mean)
    stddev = tf.sqrt(variance)
    
    return mean, stddev

def normalize_dataset(dataset, mean, stddev):
    """
    Normalize the dataset using mean and standard deviation, replacing NaN values with the mean.

    Args:
    dataset: A TensorFlow Dataset that needs to be normalized.
    mean: The mean values for each feature.
    stddev: The standard deviation for each feature.

    Returns:
    A normalized TensorFlow Dataset.
    """
    # Function to replace NaN values with the mean and normalize
    def replace_nan_and_normalize(data, target):
        # Replace NaN values with the mean
        data_no_nan = tf.where(tf.math.is_nan(data), mean, data)
        # Apply normalization using broadcasting
        normalized_data = (data_no_nan - mean) / stddev
        return normalized_data, target

    # Apply the replace and normalization function
    return dataset.map(replace_nan_and_normalize)


In [24]:
# Assuming train_dataset is a tf.data.Dataset object
mean, stddev = compute_normalization_stats(train_dataset)

# Normalize the train dataset
train_dataset = normalize_dataset(train_dataset, mean, stddev)

# Normalize the validation dataset
val_dataset = normalize_dataset(val_dataset, mean, stddev)

# Normalize the test dataset
test_dataset = normalize_dataset(test_dataset, mean, stddev)

NameError: name 'val_dataset' is not defined

Print the first record

In [66]:
# Function to print the first record in the dataset
def print_first_record(dataset, dataset_name="Dataset"):
    print(f"--- First Record from {dataset_name} ---")
    for data, label in dataset.take(1):  # Take a single batch from the dataset
        print(f"Data: {data[0]}")
        print(f"Label: {label[0]}")

# Print the first record for the train dataset
print_first_record(train_dataset, "Train Dataset")


--- First Record from Train Dataset ---
Data: [[[[ 1.00000000e+00  2.19300000e+03  2.14400000e+03 ... -2.82243614e+01
     0.00000000e+00  0.00000000e+00]
   [ 1.00000000e+00  2.19100000e+03  2.16900000e+03 ... -2.67470703e+01
     0.00000000e+00  0.00000000e+00]
   [ 1.00000000e+00  2.19200000e+03  2.19300000e+03 ... -2.46584034e+01
     0.00000000e+00  0.00000000e+00]
   ...
   [ 1.00000000e+00  1.84400000e+03  1.84100000e+03 ... -2.39988976e+01
     0.00000000e+00  0.00000000e+00]
   [ 1.00000000e+00  1.82300000e+03  1.82200000e+03 ... -2.74496632e+01
     0.00000000e+00  0.00000000e+00]
   [ 1.00000000e+00  1.80300000e+03  1.80400000e+03 ... -3.11657639e+01
     0.00000000e+00  0.00000000e+00]]

  [[ 1.00000000e+00  2.20500000e+03  2.16600000e+03 ... -2.65925369e+01
     0.00000000e+00  0.00000000e+00]
   [ 1.00000000e+00  2.20300000e+03  2.17900000e+03 ... -2.61412239e+01
     0.00000000e+00  0.00000000e+00]
   [ 1.00000000e+00  2.20100000e+03  2.19200000e+03 ... -2.55246410e+01
 

## Base training and test functions

In [11]:
# Clear any logs from previous runs
# !rm -rf ./logs/

In [26]:
%load_ext tensorboard

In [ ]:
import datetime
import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint

def train_model(model, num_epochs=50):
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Custom learning rate scheduler
    def custom_lr_scheduler(epoch, lr):
        if epoch < 10:
            return lr
        else:
            return lr * tf.math.exp(-0.1)

    # Define the path where the best model will be saved
    best_model_filepath = f"{log_dir}/best_model.h5"
    
    # ModelCheckpoint callback to save the best model based on validation accuracy
    checkpoint_callback = ModelCheckpoint(
        best_model_filepath,
        monitor='val_loss',  # Change to monitor validation accuracy
        save_best_only=True,
        mode='min',  # Change to 'max' for accuracy, since we want the highest value
        verbose=1
    )

    callbacks_list = [
        tensorboard_callback,
        LearningRateScheduler(custom_lr_scheduler, verbose=1),
        checkpoint_callback  # Add ModelCheckpoint to callbacks
    ]
    # Train the model
    history = model.fit(
        train_dataset,
        epochs=num_epochs,
        callbacks=callbacks_list,
        # validation_data=val_dataset,
    )

    # Load the best weights from the saved model
    model.load_weights(best_model_filepath)

    # Evaluate the best model on the training dataset
    train_loss, train_accuracy = model.evaluate(train_dataset, verbose=0)
    
    # Evaluate the best model on the validation dataset
    # val_loss, val_accuracy = model.evaluate(val_dataset, verbose=0)

    # Print training and validation accuracy
    print(f"Best model training accuracy: {train_accuracy:.4f}")
    # print(f"Best model validation accuracy: {val_accuracy:.4f}")


    # Return the model with the best weights
    return model


## Base 3D CNN Model

### Define the model

In [79]:
from tensorflow.keras import layers, models

def create_3d_cnn_model(input_shape):
    model = models.Sequential()

    # Input layer
    model.add(layers.InputLayer(input_shape=input_shape))

    # 3D Convolutional layers
    model.add(
        layers.Conv3D(64, kernel_size=(3, 3, 3), activation="relu", padding="same")
    )
    model.add(layers.MaxPooling3D(pool_size=(2, 2, 2)))

    model.add(
        layers.Conv3D(32, kernel_size=(3, 3, 3), activation="relu", padding="same")
    )
    model.add(layers.MaxPooling3D(pool_size=(2, 2, 2)))

    # Flatten the output to feed into a Dense layer
    model.add(layers.Flatten())

    # Dense layers
    model.add(layers.Dense(512, activation="relu"))
    model.add(layers.Dropout(0.5))  # Dropout layer to reduce overfitting
    model.add(layers.Dense(256, activation="relu"))
    model.add(layers.Dropout(0.5))  # Dropout layer to reduce overfitting
    
    # Output layer for regression (single neuron with linear activation)
    model.add(layers.Dense(1))  # No activation function for regression

    return model

# Example usage:
# input_shape = (30, 64, 64, 1)  # Adjust this based on your data
# model = create_3d_cnn_model(input_shape)

# Compile the model for regression
# model.compile(optimizer="adam", loss="mean_squared_error", metrics=["mean_absolute_error"])

# model.summary()


### Construct the model 

In [ ]:
model_3d_cnn = create_3d_cnn_model(timeseries_shape)

model_3d_cnn.compile(
    optimizer="adam", loss="mean_squared_error", metrics=["mae"]
)

model_3d_cnn.summary()

c:\Users\Surya Naganathan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d_14 (Conv3D)              │ (None, 6, 64, 64, 64)  │        25,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_14 (MaxPooling3D) │ (None, 3, 32, 32, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_15 (Conv3D)              │ (None, 3, 32, 32, 32)  │        55,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_15 (MaxPooling3D) │ (None, 1, 16, 16, 32)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 512)            │     4,194,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,407,713 (16.81 MB)

 Trainable params: 4,407,713 (16.81 MB)

 Non-trainable params: 0 (0.00 B)

### Run

In [ ]:
model_3d_cnn = train_model(model=model_3d_cnn, num_epochs=10)

test_loss, test_mae = model_3d_cnn.evaluate(test_dataset)
print(f"Test Loss: {test_loss}")
print(f"Test MAE: {test_mae}")

## CNN-LSTM

### Define the model

In [17]:
from tensorflow.keras import layers, models

def create_cnn_lstm_model(input_shape, num_classes):
    # Model definition
    model = models.Sequential()
    
    # TimeDistributed wrapper applies a layer to every temporal slice of an input.
    model.add(layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu', padding='same'), input_shape=input_shape))
    model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))
    
    model.add(layers.TimeDistributed(layers.Conv2D(32, (3, 3), activation='relu', padding='same')))
    model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))

    model.add(layers.TimeDistributed(layers.Conv2D(16, (3, 3), activation='relu', padding='same')))
    model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))
    
    # Flatten the spatial dimensions, but keep the temporal dimension
    model.add(layers.TimeDistributed(layers.Flatten()))
    
    # LSTM layer to process the temporal sequence
    model.add(layers.LSTM(256, return_sequences=False))
    
    # Dense layers for classification
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model


### Construct the model 

In [18]:

model_cnn_lstm = create_cnn_lstm_model(timeseries_shape, num_classes)

model_cnn_lstm.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

model_cnn_lstm.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDist  (None, 6, 64, 64, 64)     8704      
 ributed)                                                        
                                                                 
 time_distributed_1 (TimeDi  (None, 6, 32, 32, 64)     0         
 stributed)                                                      
                                                                 
 time_distributed_2 (TimeDi  (None, 6, 32, 32, 32)     18464     
 stributed)                                                      
                                                                 
 time_distributed_3 (TimeDi  (None, 6, 16, 16, 32)     0         
 stributed)                                                      
                                                                 
 time_distributed_4 (TimeDi  (None, 6, 16, 16, 16)    

### Train

In [19]:
model_cnn_lstm = train_model(model_cnn_lstm, 200)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/200


2024-09-19 18:04:21.550502: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


     27/Unknown - 8s 129ms/step - loss: 1.1302 - accuracy: 0.2874

2024-09-19 18:04:29.020901: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 4427342384766040563
2024-09-19 18:04:29.020911: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15259108753332222775
2024-09-19 18:04:29.020915: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 16223866063186725963
2024-09-19 18:04:29.020918: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 13538707906064767957
2024-09-19 18:04:29.020923: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 15689828834346347568
2024-09-19 18:04:29.533696: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7135953658816742753
2024-09-19 18:04:29.533711: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv


Epoch 1: val_accuracy improved from -inf to 0.34146, saving model to logs/fit/20240919-180420/best_model.h5
27/27 [==============================] - 9s 159ms/step - loss: 1.1302 - accuracy: 0.2874 - val_loss: 1.0738 - val_accuracy: 0.3415 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/200


/Users/mahdifarnaghi/workspace/dailycodes/240125_phenology_kostas/env/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


26/27 [===========================>..] - ETA: 0s - loss: 1.1156 - accuracy: 0.3654
Epoch 2: val_accuracy did not improve from 0.34146
27/27 [==============================] - 1s 41ms/step - loss: 1.1113 - accuracy: 0.3640 - val_loss: 1.0564 - val_accuracy: 0.3415 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/200
26/27 [===========================>..] - ETA: 0s - loss: 1.0912 - accuracy: 0.3615
Epoch 3: val_accuracy did not improve from 0.34146
27/27 [==============================] - 1s 40ms/step - loss: 1.0935 - accuracy: 0.3640 - val_loss: 1.0397 - val_accuracy: 0.3415 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/200
25/27 [==========================>...] - ETA: 0s - loss: 1.0761 - accuracy: 0.3720
Epoch 4: val_accuracy did not improve from 0.34146
27/27 [==============================] - 2s 44ms/step - loss: 1.0759 - accuracy: 0.3640 - val_loss: 1.0228 - val_accuracy: 0.341

2024-09-19 18:09:25.313950: I tensorflow/core/framework/local_rendezvous.cc:425] Local rendezvous send item cancelled. Key hash: 11335459269281643048
2024-09-19 18:09:25.313965: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8371882402389460458
2024-09-19 18:09:25.313978: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7135953658816742753
2024-09-19 18:09:25.313988: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17560869367484556627
2024-09-19 18:09:25.313991: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10059963876239071109


Best model training accuracy: 0.3640
Best model validation accuracy: 0.3415


2024-09-19 18:09:25.536502: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7135953658816742753
2024-09-19 18:09:25.536514: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17560869367484556627
2024-09-19 18:09:25.536518: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10059963876239071109
2024-09-19 18:09:25.536521: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8371882402389460458


In [20]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model_cnn_lstm.evaluate(test_dataset)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

9/9 [==============================] - 0s 9ms/step - loss: 1.1523 - accuracy: 0.3457
Test Loss: 1.1523404121398926
Test Accuracy: 0.34567901492118835


2024-09-19 18:09:25.969977: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7135953658816742753
2024-09-19 18:09:25.969998: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17560869367484556627
2024-09-19 18:09:25.970001: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10059963876239071109
2024-09-19 18:09:25.970005: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8371882402389460458


In [21]:
# Evaluate the model on the test dataset
train_loss, train_accuracy = model_cnn_lstm.evaluate(train_dataset)

print(f"Train Loss: {train_loss}")
print(f"Train Accuracy: {train_accuracy}")

27/27 [==============================] - 1s 24ms/step - loss: 1.1200 - accuracy: 0.3640
Train Loss: 1.119980812072754
Train Accuracy: 0.3639846742153168


2024-09-19 18:09:27.005688: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 7135953658816742753
2024-09-19 18:09:27.005703: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 17560869367484556627
2024-09-19 18:09:27.005707: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10059963876239071109
2024-09-19 18:09:27.005711: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 8371882402389460458
